In [1]:
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import r2_score, mean_squared_error

<p style="font-size: 20px"> 8. In this exercise, we will generate simulated data, and will then use
this data to perform best subset selection. </p>

(a) Use the rnorm() function to generate a predictor X of length
n = 100, as well as a noise vector ϵ of length n = 100.

In [151]:
X = np.random.randn(100)
error = np.random.randn(100)

(b) Generate a response vector Y of length n = 100 according to
the model
$$Y = \beta_{0} + \beta_{1}X + \beta_{2}X^{2} + \beta_{3}X^{3}$$
where β0, β1, β2, and β3 are constants of your choice.

In [152]:
poly = PolynomialFeatures(degree=3)
X_poly = poly.fit_transform(X.reshape(-1, 1))

In [153]:
coefficients = np.random.rand(4)

In [154]:
y = X_poly @ coefficients + error

In [155]:
linreg = LinearRegression()
linreg.fit(X_poly, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

(c) Use the regsubsets() function to perform best subset selection
in order to choose the best model containing the predictors
X,X2, . . . ,X10. What is the best model obtained according to
Cp, BIC, and adjusted R2? Show some plots to provide evidence
for your answer, and report the coefficients of the best model obtained.
Note you will need to use the data.frame() function to
create a single data set containing both X and Y .

In [156]:
poly = PolynomialFeatures(degree=10, include_bias=False)
X_poly_10 = poly.fit_transform(X.reshape(-1, 1))

In [159]:
df = pd.DataFrame(np.c_[X_poly_10, y], columns=['X'+str(i) for i in range(1, 11)] + ['Y'])
display(df)
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,Y
0,-0.672383,0.452100,-0.303984,0.204394,-0.137431,0.092406,-0.062133,0.041777,-0.028090,0.018887,0.603078
1,1.548016,2.396353,3.709593,5.742509,8.889496,13.761081,21.302372,32.976411,51.048010,79.023132,5.179037
2,-1.110409,1.233008,-1.369143,1.520308,-1.688164,1.874552,-2.081519,2.311337,-2.566529,2.849896,0.488117
3,-0.824538,0.679863,-0.560573,0.462214,-0.381113,0.314242,-0.259105,0.213642,-0.176156,0.145247,-0.707139
4,1.835461,3.368915,6.183511,11.349590,20.831724,38.235807,70.180314,128.813194,236.431531,433.960737,7.314575
...,...,...,...,...,...,...,...,...,...,...,...
95,0.948777,0.900177,0.854067,0.810319,0.768812,0.729431,0.692067,0.656617,0.622983,0.591072,0.725362
96,-0.453304,0.205485,-0.093147,0.042224,-0.019140,0.008676,-0.003933,0.001783,-0.000808,0.000366,1.182504
97,0.276735,0.076582,0.021193,0.005865,0.001623,0.000449,0.000124,0.000034,0.000010,0.000003,-0.572658
98,-0.572097,0.327295,-0.187245,0.107122,-0.061284,0.035061,-0.020058,0.011475,-0.006565,0.003756,2.125837


In [160]:
pd.DataFrame([{1:2, 3:4}, {2:4, 5:7}])

,1,3,2,5
0,2.0,4.0,NaN,NaN
1,NaN,NaN,4.0,7.0


In [161]:
result_k={}
results = []
combo = list(itertools.combinations_with_replacement(X.columns, r=2))
for features in combo:
    result_k['k'] = len(features)
    result_k["features"] = features
    results

In [194]:
def calculate_aic(y_true, y_pred, n, num_params):
	aic = n * np.log(mean_squared_error(y_true, y_pred)) + 2 * num_params
	return aic
def calculate_bic(y_true, y_pred, n, num_params):
    rss = np.sum((y_pred - y) ** 2)
    sigma2 = rss / n
    return (1 / n) * (rss + np.log(n) * num_params * sigma2)
def calculate_r2_adjusted(y_true, y_pred, n, p):
    r2 = r2_score(y_true, y_pred)
    return 1 - (1 - r2) * (n - 1) / (n - p - 1)

In [195]:
def single_model(*features, X_df=X, y=y):
    
    X = X_df[list(features)].values
    n = X.shape[0]
    p = X.shape[1]
    
    linreg = LinearRegression()
    linreg.fit(X, y)
    y_pred = linreg.predict(X)
    mse = mean_squared_error(y, y_pred)
    r2 = linreg.score(X, y)
    r2_adjusted = calculate_r2_adjusted(y, y_pred, n, p)
    rss = np.sum((y_pred - y) ** 2)
    aic = calculate_aic(y, y_pred, n, p)
    bic = calculate_bic(y, y_pred, n, p)
    
    return {"features": features, "r2": r2, "adjusted r2": r2_adjusted, "aic": aic, "bic": bic}

In [196]:
class GetBestModel():
    def __init__(self, k, X_df=X, y=y, by='r2'):
        combinations = itertools.combinations(X.columns, r=k)
        self.results_ = []
        for features in combinations:
            result_ = single_model(*features)
            self.results_.append(result_)
        self.best_result_ = sorted(self.results_, key=lambda x: x[str(by)])[-1]
        
    

In [197]:
X

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10
0,-0.672383,0.452100,-0.303984,0.204394,-0.137431,0.092406,-0.062133,0.041777,-0.028090,0.018887
1,1.548016,2.396353,3.709593,5.742509,8.889496,13.761081,21.302372,32.976411,51.048010,79.023132
2,-1.110409,1.233008,-1.369143,1.520308,-1.688164,1.874552,-2.081519,2.311337,-2.566529,2.849896
3,-0.824538,0.679863,-0.560573,0.462214,-0.381113,0.314242,-0.259105,0.213642,-0.176156,0.145247
4,1.835461,3.368915,6.183511,11.349590,20.831724,38.235807,70.180314,128.813194,236.431531,433.960737
...,...,...,...,...,...,...,...,...,...,...
95,0.948777,0.900177,0.854067,0.810319,0.768812,0.729431,0.692067,0.656617,0.622983,0.591072
96,-0.453304,0.205485,-0.093147,0.042224,-0.019140,0.008676,-0.003933,0.001783,-0.000808,0.000366
97,0.276735,0.076582,0.021193,0.005865,0.001623,0.000449,0.000124,0.000034,0.000010,0.000003
98,-0.572097,0.327295,-0.187245,0.107122,-0.061284,0.035061,-0.020058,0.011475,-0.006565,0.003756


In [198]:
#run k models which k as the number of predictors
p = X.shape[1]
best_subsets = pd.DataFrame(columns=range(5))
for i in range(1, p + 1):
    print(i)
    print(GetBestModel(k=i).best_result_.values())
    best_subsets.loc[i] = GetBestModel(k=i).best_result_.values()

1
dict_values([('X3',), 0.9446490540242054, 0.9440842484530239, -1.0626887687387931, 1.0145000249713108])
2
dict_values([('X2', 'X3'), 0.9513559927351009, 0.9503530235131442, -11.979201702919996, 0.9308227075297899])
3
dict_values([('X2', 'X3', 'X5'), 0.9542639102042668, 0.9528346573981501, -16.143294018743596, 0.912082970037164])
4
dict_values([('X1', 'X2', 'X3', 'X10'), 0.9544951979365939, 0.9525792062707662, -14.650277725320553, 0.9441883706164229])
5
dict_values([('X2', 'X3', 'X5', 'X7', 'X9'), 0.9561622231235545, 0.9538304264811903, -16.382470607737584, 0.9449716153429921])
6
dict_values([('X1', 'X2', 'X3', 'X5', 'X7', 'X9'), 0.9569154477420833, 0.9541357992093145, -16.115612092265362, 0.9634999400457107])
7
dict_values([('X1', 'X2', 'X3', 'X5', 'X7', 'X9', 'X10'), 0.9571404875006275, 0.9538794376365448, -14.639302177186604, 0.99305070275017])
8
dict_values([('X1', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9'), 0.9575927794104877, 0.9538646721059152, -13.700199283475321, 1.01678951667

In [199]:
best_subsets.columns = GetBestModel(k=10).best_result_.keys()

In [200]:
#sort by adjusted r2
best_subsets.sort_values(by=['adjusted r2', 'aic', 'bic'], ascending=False)

,features,r2,adjusted r2,aic,bic
6,"(X1, X2, X3, X5, X7, X9)",0.956915,0.954136,-16.115612,0.963500
7,"(X1, X2, X3, X5, X7, X9, X10)",0.957140,0.953879,-14.639302,0.993051
8,"(X1, X3, X4, X5, X6, X7, X8, X9)",0.957593,0.953865,-13.700199,1.016790
5,"(X2, X3, X5, X7, X9)",0.956162,0.953830,-16.382471,0.944972
9,"(X1, X3, X4, X5, X6, X7, X8, X9, X10)",0.957622,0.953384,-11.767961,1.050296
10,"(X1, X2, X3, X4, X5, X6, X7, X8, X9, X10)",0.957622,0.952860,-9.768630,1.084484
3,"(X2, X3, X5)",0.954264,0.952835,-16.143294,0.912083
4,"(X1, X2, X3, X10)",0.954495,0.952579,-14.650278,0.944188
2,"(X2, X3)",0.951356,0.950353,-11.979202,0.930823
1,"(X3,)",0.944649,0.944084,-1.062689,1.014500


In [201]:
#sort by aic
best_subsets.sort_values(by=['aic'], ascending=True)

,features,r2,adjusted r2,aic,bic
5,"(X2, X3, X5, X7, X9)",0.956162,0.953830,-16.382471,0.944972
3,"(X2, X3, X5)",0.954264,0.952835,-16.143294,0.912083
6,"(X1, X2, X3, X5, X7, X9)",0.956915,0.954136,-16.115612,0.963500
4,"(X1, X2, X3, X10)",0.954495,0.952579,-14.650278,0.944188
7,"(X1, X2, X3, X5, X7, X9, X10)",0.957140,0.953879,-14.639302,0.993051
8,"(X1, X3, X4, X5, X6, X7, X8, X9)",0.957593,0.953865,-13.700199,1.016790
2,"(X2, X3)",0.951356,0.950353,-11.979202,0.930823
9,"(X1, X3, X4, X5, X6, X7, X8, X9, X10)",0.957622,0.953384,-11.767961,1.050296
10,"(X1, X2, X3, X4, X5, X6, X7, X8, X9, X10)",0.957622,0.952860,-9.768630,1.084484
1,"(X3,)",0.944649,0.944084,-1.062689,1.014500


In [202]:
best_subsets.sort_values(by=['bic'], ascending=True)

,features,r2,adjusted r2,aic,bic
3,"(X2, X3, X5)",0.954264,0.952835,-16.143294,0.912083
2,"(X2, X3)",0.951356,0.950353,-11.979202,0.930823
4,"(X1, X2, X3, X10)",0.954495,0.952579,-14.650278,0.944188
5,"(X2, X3, X5, X7, X9)",0.956162,0.953830,-16.382471,0.944972
6,"(X1, X2, X3, X5, X7, X9)",0.956915,0.954136,-16.115612,0.963500
7,"(X1, X2, X3, X5, X7, X9, X10)",0.957140,0.953879,-14.639302,0.993051
1,"(X3,)",0.944649,0.944084,-1.062689,1.014500
8,"(X1, X3, X4, X5, X6, X7, X8, X9)",0.957593,0.953865,-13.700199,1.016790
9,"(X1, X3, X4, X5, X6, X7, X8, X9, X10)",0.957622,0.953384,-11.767961,1.050296
10,"(X1, X2, X3, X4, X5, X6, X7, X8, X9, X10)",0.957622,0.952860,-9.768630,1.084484


9. In this exercise, we will predict the number of applications received
using the other variables in the College data set.

In [55]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [56]:
pwd

'C:\\Users\\lamtr\\Introduction to Statistical Learning\\6. Linear Model Selection and Regularization'

In [57]:
college = pd.read_csv('../datasets/College.csv')

In [58]:
college.head()

,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
0,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
1,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
2,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
3,Yes,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
4,Yes,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15


In [59]:
college["Private"] = college["Private"].replace(to_replace={"Yes": 1, "No": 0})

In [60]:
college.describe()

,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
count,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.00000
mean,0.727156,3001.638353,2018.804376,779.972973,27.558559,55.796654,3699.907336,855.298584,10440.669241,4357.526384,549.380952,1340.642214,72.660232,79.702703,14.089704,22.743887,9660.171171,65.46332
std,0.445708,3870.201484,2451.113971,929.176190,17.640364,19.804778,4850.420531,1522.431887,4023.016484,1096.696416,165.105360,677.071454,16.328155,14.722359,3.958349,12.391801,5221.768440,17.17771
min,0.000000,81.000000,72.000000,35.000000,1.000000,9.000000,139.000000,1.000000,2340.000000,1780.000000,96.000000,250.000000,8.000000,24.000000,2.500000,0.000000,3186.000000,10.00000
25%,0.000000,776.000000,604.000000,242.000000,15.000000,41.000000,992.000000,95.000000,7320.000000,3597.000000,470.000000,850.000000,62.000000,71.000000,11.500000,13.000000,6751.000000,53.00000
50%,1.000000,1558.000000,1110.000000,434.000000,23.000000,54.000000,1707.000000,353.000000,9990.000000,4200.000000,500.000000,1200.000000,75.000000,82.000000,13.600000,21.000000,8377.000000,65.00000
75%,1.000000,3624.000000,2424.000000,902.000000,35.000000,69.000000,4005.000000,967.000000,12925.000000,5050.000000,600.000000,1700.000000,85.000000,92.000000,16.500000,31.000000,10830.000000,78.00000
max,1.000000,48094.000000,26330.000000,6392.000000,96.000000,100.000000,31643.000000,21836.000000,21700.000000,8124.000000,2340.000000,6800.000000,103.000000,100.000000,39.800000,64.000000,56233.000000,118.00000


In [61]:
college.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 777 entries, 0 to 776
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Private      777 non-null    int64  
 1   Apps         777 non-null    int64  
 2   Accept       777 non-null    int64  
 3   Enroll       777 non-null    int64  
 4   Top10perc    777 non-null    int64  
 5   Top25perc    777 non-null    int64  
 6   F.Undergrad  777 non-null    int64  
 7   P.Undergrad  777 non-null    int64  
 8   Outstate     777 non-null    int64  
 9   Room.Board   777 non-null    int64  
 10  Books        777 non-null    int64  
 11  Personal     777 non-null    int64  
 12  PhD          777 non-null    int64  
 13  Terminal     777 non-null    int64  
 14  S.F.Ratio    777 non-null    float64
 15  perc.alumni  777 non-null    int64  
 16  Expend       777 non-null    int64  
 17  Grad.Rate    777 non-null    int64  
dtypes: float64(1), int64(17)
memory usage: 109.4 KB


In [62]:
college.dtypes

Private          int64
Apps             int64
Accept           int64
Enroll           int64
Top10perc        int64
Top25perc        int64
F.Undergrad      int64
P.Undergrad      int64
Outstate         int64
Room.Board       int64
Books            int64
Personal         int64
PhD              int64
Terminal         int64
S.F.Ratio      float64
perc.alumni      int64
Expend           int64
Grad.Rate        int64
dtype: object

In [63]:
college["Private"].value_counts()

1    565
0    212
Name: Private, dtype: int64

(a) Split the data set into a training set and a test set

In [152]:
X = college[college.columns.difference(["Apps"])]
y = college.loc[:, "Apps"]

In [153]:
X_train, X_test,  y_train, y_test = train_test_split(X, y)

(b) Fit a linear model using least squares on the training set, and
report the test error obtained.

In [66]:
linreg = LinearRegression()

In [67]:
linreg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [68]:
print("Training R2:", linreg.score(X_train, y_train))
print("Test R2:", linreg.score(X_test, y_test))

Training R2: 0.9348921626903748
Test R2: 0.896808728897916


In [69]:
def r2_adjusted(y_true, y_pred, m, n):
    from sklearn.metrics import r2_score
    return 1 - (1 - r2_score(y_true, y_pred) * (m - 1) / (m - n - 1))

In [70]:
m = X.shape[0]
n = X.shape[1]
training_y_pred = linreg.predict(X_train)
r2_adjusted(y_train, training_y_pred, m, n)

0.9558317763474715

In [71]:
m = X_train.shape[0]
n = X_train.shape[1]
training_y_pred = linreg.predict(X_train)
r2_adjusted(y_train, training_y_pred, m, n)

0.963071536388489

In [72]:
m = X_test.shape[0]
n = X_test.shape[1]
test_y_pred = linreg.predict(X_test)
r2_adjusted(y_test, test_y_pred, m, n)

0.9829429005999757

In [73]:
print("Training MSE:", mean_squared_error(y_train, training_y_pred))

Training MSE: 1064675.624039492


In [74]:
print("Test MSE:", mean_squared_error(y_test, test_y_pred))

Test MSE: 1108041.8239321497


In [83]:
' + '.join(college.columns.difference(["Apps"]))

'Accept + Books + Enroll + Expend + F.Undergrad + Grad.Rate + Outstate + P.Undergrad + Personal + PhD + Private + Room.Board + S.F.Ratio + Terminal + Top10perc + Top25perc + perc.alumni'

In [86]:
college

,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
0,1,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
1,1,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
2,1,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
3,1,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
4,1,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772,0,2197,1515,543,4,26,3089,2029,6797,3900,500,1200,60,60,21.0,14,4469,40
773,1,1959,1805,695,24,47,2849,1107,11520,4960,600,1250,73,75,13.3,31,9189,83
774,1,2097,1915,695,34,61,2793,166,6900,4200,617,781,67,75,14.4,20,8323,49
775,1,10705,2453,1317,95,99,5217,83,19840,6510,630,2115,96,96,5.8,49,40386,99


In [104]:
import statsmodels.formula.api as smf
import statsmodels.api as sm


In [139]:
X = college[college.columns.difference(["Apps", "Books", "perc.alumni", "Personal", "S.F.Ratio", "P.Undergrad", "Terminal"])]
y = college.loc[:, "Apps"]

In [140]:
X_train, X_test,  y_train, y_test = train_test_split(X, y)

In [141]:
est = sm.OLS(y, X).fit()

In [148]:
est.rsquared

0.955549392727859

In [142]:
est.rsquared_adj

0.9549110680803479

In [143]:
est.mse_total

23969015.033462033

In [144]:
est.mse_model

1617825447.1944487

In [145]:

est.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Accept,1.5883,0.039,40.886,0.000,1.512,1.665
Enroll,-0.9110,0.183,-4.972,0.000,-1.271,-0.551
Expend,0.0728,0.011,6.405,0.000,0.050,0.095
F.Undergrad,0.0710,0.031,2.273,0.023,0.010,0.132
Grad.Rate,7.5259,2.671,2.818,0.005,2.283,12.769
Outstate,-0.0888,0.018,-4.951,0.000,-0.124,-0.054
PhD,-11.1783,2.666,-4.192,0.000,-16.413,-5.944
Private,-541.5673,128.101,-4.228,0.000,-793.038,-290.097
Room.Board,0.1474,0.044,3.353,0.001,0.061,0.234
Top10perc,50.9191,4.942,10.302,0.000,41.217,60.622


In [160]:
from sklearn.linear_model import Lasso, LassoCV
from sklearn.model_selection import RepeatedKFold
lasso = Lasso()
lasso.fit(X_train, y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [155]:
lasso.score(X_train, y_train)

0.9410321117488498

In [156]:
lasso.score(X_test, y_test)

0.8971701609874931

In [157]:
mean_squared_error(y_train, lasso.predict(X_train))

732465.9852029647

In [158]:
mean_squared_error(y_test, lasso.predict(X_test))

2317046.5690483893

In [162]:
np.logspace(-3, 3, num=len(np.arange(-3, 4)))

array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])

In [172]:
X.values

array([[1232.,  450.,  721., ...,   23.,   52.,   12.],
       [1924.,  750.,  512., ...,   16.,   29.,   16.],
       [1097.,  400.,  336., ...,   22.,   50.,   30.],
       ...,
       [1915.,  617.,  695., ...,   34.,   61.,   20.],
       [2453.,  630., 1317., ...,   95.,   99.,   49.],
       [1855.,  500.,  691., ...,   28.,   63.,   28.]])

In [177]:
X.values

array([[1232.,  450.,  721., ...,   23.,   52.,   12.],
       [1924.,  750.,  512., ...,   16.,   29.,   16.],
       [1097.,  400.,  336., ...,   22.,   50.,   30.],
       ...,
       [1915.,  617.,  695., ...,   34.,   61.,   20.],
       [2453.,  630., 1317., ...,   95.,   99.,   49.],
       [1855.,  500.,  691., ...,   28.,   63.,   28.]])

In [218]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
lassoCV = LassoCV(alphas=np.logspace(-3, 3, num=len(np.arange(-3, 4))), cv=cv)
lassoCV.fit(X.values, y.values.reshape(-1, 1))

C:\Users\lamtr\anaconda3\envs\clone_3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LassoCV(alphas=array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
        copy_X=True, cv=RepeatedKFold(n_repeats=3, n_splits=10, random_state=1),
        eps=0.001, fit_intercept=True, max_iter=1000, n_alphas=100, n_jobs=None,
        normalize=False, positive=False, precompute='auto', random_state=None,
        selection='cyclic', tol=0.0001, verbose=False)

In [219]:
lassoCV.score(X_train, y_train)

0.9370154682940025

In [220]:
lassoCV.mse_path_.mean(axis=1)

array([1322554.01811157, 1294350.22357601, 1286128.75695257,
       1286433.48598025, 1286595.29596093, 1286612.78776948,
       1286614.55009407])

In [221]:
lassoCV.alphas_

array([1.e+03, 1.e+02, 1.e+01, 1.e+00, 1.e-01, 1.e-02, 1.e-03])

In [222]:
lassoCV.alpha_

10.0

In [223]:
lassoCV.mse_path_.mean(axis=1)

array([1322554.01811157, 1294350.22357601, 1286128.75695257,
       1286433.48598025, 1286595.29596093, 1286612.78776948,
       1286614.55009407])